In [113]:
import pandas as pd
import datetime
import os

In [2]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

In [3]:
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="6bf9b68f-809e-4986-a474-300c6ca2eaa9",
    resource_group_name="mcdiismedinarg001",
    workspace_name="mcdiaml001",
)

In [4]:
from azure.ai.ml.entities import AmlCompute

gpu_compute_target = "gpu-cluster"

try:
    # let's see if the compute target already exists
    gpu_cluster = ml_client.compute.get(gpu_compute_target)
    print(
        f"You already have a cluster named {gpu_compute_target}, we'll reuse it as is."
    )

except Exception:
    print("Creating a new gpu compute target...")

    # Let's create the Azure ML compute object with the intended parameters
    gpu_cluster = AmlCompute(
        # Name assigned to the compute cluster
        name="gpu-cluster",
        # Azure ML Compute is the on-demand VM service
        type="amlcompute",
        # VM Family
        size="STANDARD_NC6",
        # Minimum running nodes when there is no job running
        min_instances=0,
        # Nodes in cluster
        max_instances=4,
        # How many seconds will the node running after the job termination
        idle_time_before_scale_down=180,
        # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
        tier="Dedicated",
    )

    # Now, we pass the object to MLClient's create_or_update method
    gpu_cluster = ml_client.begin_create_or_update(gpu_cluster).result()

print(
    f"AMLCompute with name {gpu_cluster.name} is created, the compute size is {gpu_cluster.size}"
)

You already have a cluster named gpu-cluster, we'll reuse it as is.
AMLCompute with name gpu-cluster is created, the compute size is STANDARD_NC6


In [5]:
curated_env_name = "AzureML-tensorflow-2.7-ubuntu20.04-py38-cuda11-gpu@latest"

In [6]:
##web_path = "wasbs://datasets@azuremlexamples.blob.core.windows.net/mnist/"

In [30]:
from azure.ai.ml import command
from azure.ai.ml import UserIdentityConfiguration
from azure.ai.ml import Input

web_path = "azureml://subscriptions/6bf9b68f-809e-4986-a474-300c6ca2eaa9/resourcegroups/mcdiismedinarg001/workspaces/mcdiaml001/datastores/workspaceblobstore/paths/mcdi2021-proyecto/"

job = command(
    inputs=dict(
        data_folder=Input(type="uri_folder", path=web_path),
        batch_size=64,
        first_layer_neurons=256,
        second_layer_neurons=128,
        learning_rate=0.01,
    ),
    compute=gpu_compute_target,
    environment=curated_env_name,
    code="./src/",
    command="python tf_train.py --data-folder ${{inputs.data_folder}} --batch-size ${{inputs.batch_size}} --first-layer-neurons ${{inputs.first_layer_neurons}} --second-layer-neurons ${{inputs.second_layer_neurons}} --learning-rate ${{inputs.learning_rate}}",
    experiment_name="tf-pianojazzmusic-prediction",
    display_name="tensorflow-pianojazzmusic-prediction-with-rnn",
)

In [31]:
job_tracker = ml_client.jobs.create_or_update(job)

In [40]:
job_tracker.name
best_run = job_tracker.name
"azureml://jobs/{}/outputs/artifacts/paths/outputs/model/".format(
        best_run
    )

'azureml://jobs/olden_tent_2fgjdpq7d3/outputs/artifacts/paths/outputs/model/'

In [41]:
from azure.ai.ml.entities import Model

best_run = job_tracker.name

# lets get the model from this run
model = Model(
    # the script stores the model as "model"
    path="azureml://jobs/{}/outputs/artifacts/paths/outputs/model/".format(
        best_run
    ),
    name="mcdi2021_prod_model",
    description="Model created from run.",
    type="custom_model",
)

In [42]:
registered_model = ml_client.models.create_or_update(model=model)

In [43]:
import uuid

# Creating a unique name for the endpoint
online_endpoint_name = "mcdi2021-rnn-endpoint-" + str(uuid.uuid4())[:8]

In [44]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Predicts jazz piano music sequence using recurrent neural network (RNN) using TensorFlow",
    auth_mode="key",
)

endpoint = ml_client.begin_create_or_update(endpoint).result()

print(f"Endpint {endpoint.name} provisioning state: {endpoint.provisioning_state}")

Endpint mcdi2021-rnn-endpoint-20c1d0d5 provisioning state: Succeeded


In [45]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

Endpint "mcdi2021-rnn-endpoint-20c1d0d5" with provisioning state "Succeeded" is retrieved


In [101]:
model = registered_model

from azure.ai.ml.entities import CodeConfiguration

# create an online deployment.
blue_deployment = ManagedOnlineDeployment(
    name="tff-blue",
    endpoint_name=online_endpoint_name,
    model=model,
    code_configuration=CodeConfiguration(code="./src", scoring_script="score.py"),
    environment=curated_env_name,
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

blue_deployment = ml_client.begin_create_or_update(blue_deployment).result()

Check: endpoint mcdi2021-rnn-endpoint-20c1d0d5 exists
Uploading src (0.03 MBs): 100%|██████████| 31781/31781 [00:00<00:00, 1085462.81it/s]


data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


................................

In [123]:
import json
import numpy as np

time_stamp = datetime.datetime.now().strftime("%Y%m%d_%H_%M_%S_%f")
folder = './output/AzureML_' + time_stamp
if not os.path.exists(folder):
    os.makedirs(folder)


n_notes = [71, 0, 0, 0 ,0, 36, 48, 0, 0, 0]

test_samples = json.dumps({"data": n_notes})

json_file_request = folder + "/sample-request.json"

with open(json_file_request, "w") as outfile:
    outfile.write(test_samples)

In [124]:
# # predict using the deployed model
result = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file=json_file_request,
    deployment_name="tff-blue",
)

In [125]:
import json

music_created = json.loads(result)

In [126]:
n_notes_4_4_sequence = pd.json_normalize(music_created['n_notes_4_4_sequence'])
n_notes_2_4_sequence = pd.json_normalize(music_created['n_notes_2_4_sequence'])
n_notes_2_2_sequence = pd.json_normalize(music_created['n_notes_2_2_sequence'])
n_notes_3_4_sequence = pd.json_normalize(music_created['n_notes_3_4_sequence'])
n_notes_5_4_sequence = pd.json_normalize(music_created['n_notes_5_4_sequence'])

In [128]:
n_notes_4_4_sequence.to_csv(folder + '/n_notes_4_4_sequence.csv')
n_notes_2_4_sequence.to_csv(folder + '/n_notes_2_4_sequence.csv')
n_notes_2_2_sequence.to_csv(folder + '/n_notes_2_2_sequence.csv')
n_notes_3_4_sequence.to_csv(folder + '/n_notes_3_4_sequence.csv')
n_notes_5_4_sequence.to_csv(folder + '/n_notes_5_4_sequence.csv')